<a href="https://colab.research.google.com/github/UznetDev/Data-science-home-work/blob/main/07_Avg_2024_home_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import *
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, median_absolute_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split, RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import RFE, RFECV
from sklearn.ensemble import StackingRegressor, VotingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
import warnings
import re
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('car_prices_clean.csv')
df.head(1)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,...,brand_rank,model_rank,fuel_type_rank,engine_rank,transmission_rank,ext_col_rank,int_col_rank,accident_rank,motor_rank,fuel_rank
0,Ford,F-150 Lariat,2018,74349,Gasoline,375.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,Blue,Gray,None reported,...,31,1256,4,821,28,91,14,2,1,3


In [3]:
columns = ['model_year',
           'milage',
           'hp',
           'litr',
           'Cylinder',
           'brand_rank',
           'model_rank',
           'fuel_type_rank',
           'engine_rank',
           'transmission_rank',
           'ext_col_rank',
           'int_col_rank',
           'accident_rank',
           'motor_rank',
           'fuel_rank'
           ]

In [4]:
X = df[columns]
y = df['price']
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = pd.DataFrame(poly.fit_transform(X), columns=poly.get_feature_names_out(X.columns))

In [5]:
# cv = KFold(n_splits=6, shuffle=True, random_state=42)

# linear_model = LinearRegression()

# rfecv = RFECV(estimator=linear_model, step=1, cv=cv, scoring='neg_mean_squared_error', verbose=2)

# rfecv.fit(X_poly, y)

best_columns = ['model_year',
 'milage',
 'hp',
 'litr',
 'Cylinder',
 'brand_rank',
 'model_rank',
 'fuel_type_rank',
 'engine_rank',
 'transmission_rank',
 'ext_col_rank',
 'int_col_rank',
 'accident_rank',
 'motor_rank',
 'fuel_rank',
 'model_year^2',
 'model_year milage',
 'model_year hp',
 'model_year litr',
 'model_year Cylinder',
 'model_year brand_rank',
 'model_year model_rank',
 'model_year fuel_type_rank',
 'model_year engine_rank',
 'model_year transmission_rank',
 'model_year ext_col_rank',
 'model_year int_col_rank',
 'model_year accident_rank',
 'model_year motor_rank',
 'model_year fuel_rank',
 'milage^2',
 'milage hp',
 'milage litr',
 'milage Cylinder',
 'milage brand_rank',
 'milage model_rank',
 'milage fuel_type_rank',
 'milage engine_rank',
 'milage transmission_rank',
 'milage ext_col_rank',
 'milage int_col_rank',
 'milage accident_rank',
 'milage motor_rank',
 'milage fuel_rank',
 'hp^2',
 'hp litr',
 'hp Cylinder',
 'hp brand_rank',
 'hp model_rank',
 'hp fuel_type_rank',
 'hp engine_rank',
 'hp transmission_rank',
 'hp ext_col_rank',
 'hp int_col_rank',
 'hp accident_rank',
 'hp motor_rank',
 'hp fuel_rank',
 'litr^2',
 'litr Cylinder',
 'litr brand_rank',
 'litr model_rank',
 'litr fuel_type_rank',
 'litr engine_rank',
 'litr transmission_rank',
 'litr ext_col_rank',
 'litr int_col_rank',
 'litr accident_rank',
 'litr motor_rank',
 'litr fuel_rank',
 'Cylinder^2',
 'Cylinder brand_rank',
 'Cylinder model_rank',
 'Cylinder fuel_type_rank',
 'Cylinder engine_rank',
 'Cylinder transmission_rank',
 'Cylinder ext_col_rank',
 'Cylinder int_col_rank',
 'Cylinder accident_rank',
 'Cylinder motor_rank',
 'Cylinder fuel_rank',
 'brand_rank^2',
 'brand_rank model_rank',
 'brand_rank fuel_type_rank',
 'brand_rank engine_rank',
 'brand_rank transmission_rank',
 'brand_rank ext_col_rank',
 'brand_rank int_col_rank',
 'brand_rank accident_rank',
 'brand_rank motor_rank',
 'brand_rank fuel_rank',
 'model_rank^2',
 'model_rank fuel_type_rank',
 'model_rank engine_rank',
 'model_rank transmission_rank',
 'model_rank ext_col_rank',
 'model_rank int_col_rank',
 'model_rank accident_rank',
 'model_rank motor_rank',
 'model_rank fuel_rank',
 'fuel_type_rank^2',
 'fuel_type_rank engine_rank',
 'fuel_type_rank transmission_rank',
 'fuel_type_rank ext_col_rank',
 'fuel_type_rank int_col_rank',
 'fuel_type_rank accident_rank',
 'fuel_type_rank motor_rank',
 'fuel_type_rank fuel_rank',
 'engine_rank^2',
 'engine_rank transmission_rank',
 'engine_rank ext_col_rank',
 'engine_rank int_col_rank',
 'engine_rank accident_rank',
 'engine_rank motor_rank',
 'engine_rank fuel_rank',
 'transmission_rank^2',
 'transmission_rank ext_col_rank',
 'transmission_rank int_col_rank',
 'transmission_rank accident_rank',
 'transmission_rank motor_rank',
 'transmission_rank fuel_rank',
 'ext_col_rank^2',
 'ext_col_rank int_col_rank',
 'ext_col_rank accident_rank',
 'ext_col_rank motor_rank',
 'ext_col_rank fuel_rank',
 'int_col_rank^2',
 'int_col_rank accident_rank',
 'int_col_rank motor_rank',
 'int_col_rank fuel_rank',
 'accident_rank^2',
 'accident_rank motor_rank',
 'accident_rank fuel_rank',
 'motor_rank^2',
 'motor_rank fuel_rank',
 'fuel_rank^2']

In [6]:
# list(X_poly.columns[rfecv.support_])

In [7]:
# optimal_num_features = rfecv.n_features_
# X_select = X_poly[X_poly.columns[rfecv.support_]]
X_select = X_poly[best_columns]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_select, y, test_size=0.2, random_state=42)

In [9]:
estimators = [
    ('lr', LinearRegression()),
    ('ridge', Ridge()),
    ('lasso', Lasso()),
    ('elasticnet', ElasticNet())
]

In [ ]:
stacking_model = StackingRegressor(estimators=estimators, final_estimator=LinearRegression())

param_distributions = {
    'ridge__alpha': np.logspace(-3, 3, 10),
    'lasso__alpha': np.logspace(-3, 3, 10),
    'elasticnet__alpha': np.logspace(-3, 3, 10),
    'elasticnet__l1_ratio': np.linspace(0.1, 0.9, 9)
}

random_search = RandomizedSearchCV(stacking_model,
                                   param_distributions,
                                   cv=5,
                                   scoring='neg_mean_squared_error',
                                  #  random_state=42,
                                   n_jobs=-1)
random_search.fit(X_train, y_train)

In [ ]:
best_params_random = random_search.best_params_
rmse = np.sqrt(-random_search.best_score_)
print("Best params:", best_params_random)
print("Best score:", rmse)

In [ ]:
# param_grid = {
#     'ridge__alpha': np.linspace(best_params_random['ridge__alpha']*0.8, best_params_random['ridge__alpha']*1.2, 5),
#     'lasso__alpha': np.linspace(best_params_random['lasso__alpha']*0.8, best_params_random['lasso__alpha']*1.2, 5),
#     'elasticnet__alpha': np.linspace(best_params_random['elasticnet__alpha']*0.8, best_params_random['elasticnet__alpha']*1.2, 5),
#     'elasticnet__l1_ratio': np.linspace(best_params_random['elasticnet__l1_ratio']*0.8, best_params_random['elasticnet__l1_ratio']*1.2, 5)
# }


param_grid = {
    'ridge__alpha': [best_params_random['ridge__alpha'] * 0.9, best_params_random['ridge__alpha'] * 1.1],
    'lasso__alpha': [best_params_random['lasso__alpha'] * 0.9, best_params_random['lasso__alpha'] * 1.1],
    'elasticnet__alpha': [best_params_random['elasticnet__alpha'] * 0.9, best_params_random['elasticnet__alpha'] * 1.1],
    'elasticnet__l1_ratio': [best_params_random['elasticnet__l1_ratio'] * 0.9, best_params_random['elasticnet__l1_ratio'] * 1.1]
}

In [ ]:
grid_search = GridSearchCV(random_search.best_estimator_,
                           param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           n_jobs=-1)
grid_search.fit(X_train, y_train)

In [ ]:
best_params_grid = grid_search.best_params_
print("Best param GridSearchCV:", best_params_grid)

In [ ]:
y_pred = grid_search.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



# estimators = [
#     ('lr', Pipeline([
#         ('scaler', StandardScaler()),
#         ('lr', LinearRegression())
#     ])),
#     ('ridge', Pipeline([
#         ('scaler', StandardScaler()),
#         ('ridge', Ridge())
#     ])),
#     ('lasso', Pipeline([
#         ('scaler', StandardScaler()),
#         ('lasso', Lasso())
#     ])),
#     ('elasticnet', Pipeline([
#         ('scaler', StandardScaler()),
#         ('elasticnet', ElasticNet())
#     ]))
# ]

# param_grid = {
#     'final_estimator__alpha': [0.1, 0.5, 1, 5, 10, 100],  # Ridge uchun
#     'ridge__ridge__alpha': [0.1, 1, 10],  # Ridge modelining alpha parametri
#     'lasso__lasso__alpha': [0.01, 0.1, 1],  # Lasso modelining alpha parametri
#     'elasticnet__elasticnet__alpha': [0.01, 0.1, 1],  # ElasticNet modelining alpha parametri
#     'elasticnet__elasticnet__l1_ratio': [0.1, 0.5, 0.9]  # ElasticNet modelining l1_ratio parametri
# }

# stacking_model = StackingRegressor(estimators=estimators, final_estimator=Ridge())

# random_search = RandomizedSearchCV(stacking_model,
#                                    param_distributions=param_grid,
#                                   #  n_iter=10,
#                                   #  cv=5,
#                                   #  random_state=0
#                                    )


# random_search.fit(X_train, y_train)


# random_search.best_params_